In [ ]:
import matplotlib.pyplot as plt
from quantopian.research.experimental import continuous_future, history
import pandas as pd
import numpy as np

### 5限月分の終値を取得

In [ ]:
def get_fut_hist(symbol, start, end):
    df = history(
        [continuous_future(symbol, offset=i,  adjustment=None) for i in range(5)],
        fields='price', 
        frequency='daily', 
        start=start, 
        end=end
    )

    df.columns = ['f{}'.format(i) for i in range(1, 6)]
    return df

In [ ]:
cl = get_fut_hist('CL', '2015-01-01', '2018-6-15')
ho = get_fut_hist('HO', '2015-01-01', '2018-6-15')
xb = get_fut_hist('XB', '2015-01-01', '2018-6-15')
ng = get_fut_hist('ng', '2015-01-01', '2018-6-15')

In [ ]:
cl.head()

In [ ]:
my_cl = cl.copy()
my_cl["remain days"] = cl_pricing["remain_date"]
my_cl["contango1"] = my_cl["f2"] / my_cl["f1"]
my_cl["contango2"] = my_cl["f3"] / my_cl["f2"]
my_cl["contango3"] = my_cl["f3"] / my_cl["f1"]


In [ ]:

def get_expiration_date(contract):
      return contract.expiration_date

def insert_expiration_date(symbol, start, end ):
    contract = history(continuous_future(symbol), fields=['contract'],frequency="daily", start=start, end=end)
    contract["expiration_date"] = contract['contract'].apply(get_expiration_date)
    contract["date"] = contract.index # memo1
    contract["remain_date"] = (contract["expiration_date"] - contract["date"]).dt.days
    return contract
    
cl_contract = insert_expiration_date('CL', '2015-01-01', '2018-6-15')
ho_contract = insert_expiration_date('HO', '2015-01-01', '2018-6-15')
xb_contract = insert_expiration_date('XB', '2015-01-01', '2018-6-15')
ng_contract = insert_expiration_date('NG', '2015-01-01', '2018-6-15')


In [ ]:
ng_contract.head()

In [ ]:
my_cl[['contango1', 'contango2', ]].plot()

In [ ]:
import numpy as np
(my_cl['contango1'] - my_cl['contango2']).apply(np.abs).describe()

In [ ]:
my_cl["spread wide"] = ((my_cl['contango1'] - my_cl['contango2']).apply(np.abs) > 0.01) #& my_cl["remain days"] > 18
#my_cl["remain days"] > 18
my_cl[my_cl["spread wide"] & (my_cl["remain days"] > 18)]

In [ ]:
x = my_cl[['f1','f2','f3']].loc["2015-04-23":].head(20)
x.plot(marker='o')
(x['f3']-x['f1']).plot(secondary_y=True)

x.iloc[-1]/x.iloc[0]-1

In [ ]:
def plot_forward(df, rows, cols, **kwargs):
    n = rows * cols
    if 'figsize' in kwargs:
        fig = plt.figure(figsize=kwargs['figsize'])
    else:
        fig = plt.figure()
    if 'suptitle' in kwargs:
        fig.suptitle(kwargs['suptitle'])
    ax = [fig.add_subplot(rows, cols, i) for i in range(1, n + 1)]
    for i, ax_ in enumerate(ax):
        ax_.plot(df.iloc[-n + i].values, marker='o')
        ax_.set_title('{:%m-%d}'.format(df.index[-n + i]))
        ax_.axes.set_xticks([])
        ax_.axes.set_yticks([])

def plot_forward_1(df, rows, cols, df_contract, **kwargs):
    n = rows * cols
    if 'figsize' in kwargs:
        fig = plt.figure(figsize=kwargs['figsize'])
    else:
        fig = plt.figure()
    if 'suptitle' in kwargs:
        fig.suptitle(kwargs['suptitle'])
    ax = [fig.add_subplot(rows, cols, i) for i in range(1, n + 1)]
    for i, ax_ in enumerate(ax):
        ax_.plot(df.iloc[-n + i].values, marker='o')
        ax_.set_title('{:%y-%m-%d} ({})'.format(df.index[-n + i], df_contract['remain_date'][-n + i] ))
        ax_.axes.set_xticks([])
        ax_.axes.set_yticks([])        

In [ ]:
plot_forward_1(cl, 30, 8, cl_contract, figsize=(14, 22), suptitle='CL')
#plot_forward(cl, 20, 8, figsize=(14, 22), suptitle='CL')

In [ ]:
xb.loc["2017-11-1":].head()

In [ ]:
plot_forward_1(xb.loc["2017-11-1":].head(50),10, 5, ho_contract, figsize=(14, 22), suptitle='xb')
#plot_forward(ho, 20, 8, figsize=(14, 22), suptitle='HO')

In [ ]:
## target DEBESO
## f2 と f4 の平均値が，f3と比べてどのくらいずれているかを調べる
## f1,2,3でDEBESOを見つけるのが，良いように見える

def debeso_checker(data, target1, target2, target3, **kwargs):
    df = data[[target1, target2, target3]]
    df['mean'] = (df[target1] + df[target3])/2
    df['diff'] = (df[target2]/df['mean'])
    df['diff'].hist(bins=100,  alpha=kwargs['alpha'], color=kwargs['color'])
    print(df.describe())

debeso_checker(cl, 'f1','f2','f3', alpha=0.7, color='blue')
debeso_checker(cl, 'f2','f3', 'f4', alpha=0.6, color='green')
debeso_checker(cl, 'f3', 'f4', 'f5', alpha=0.4, color='purple')

In [ ]:
df = pd.concat([ho, ho_contract], axis=1)
df = df[['f1', 'f2', 'f3', 'remain_date']]
df['mean'] = (df['f1'] + df['f3'])/2
df['diff'] = df['f2']/df['mean']
#df['deco'] = df['diff'] > 1.0006
df['boco'] = df['diff'] < 0.9982
df.loc["2015-2-1":]

In [ ]:
plot_forward_1(xb, 20, 8, xb_contract, figsize=(14, 22), suptitle='XB')
#plot_forward(xb, 20, 8, figsize=(14, 22), suptitle='XB')

In [ ]:
plot_forward(ng, 20, 8, figsize=(14, 22), suptitle='NG')

In [ ]:
pd.concat([xb, xb_contract["expiration_date"]], axis=1)

In [ ]:
grp = pd.concat([xb, xb_contract["expiration_date"]], axis=1).groupby('expiration_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc["std"].plot(kind='bar')
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc['50%'].plot(secondary_y=True, c='red')
#pd.concat([xb, xb_contract["expiration_date"]], axis=1).describe().loc['std'].plot(kind='bar')

In [ ]:
grp = pd.concat([ho, ho_contract["expiration_date"]], axis=1).groupby('expiration_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc["std"].plot(kind='bar')
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc['50%'].plot(secondary_y=True, c='red')

In [ ]:
grp = pd.concat([cl, cl_contract["expiration_date"]], axis=1).groupby('expiration_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc["std"].plot(kind='bar')
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc['50%'].plot(secondary_y=True, c='red')


In [ ]:
grp = pd.concat([ng, ng_contract["expiration_date"]], axis=1).groupby('expiration_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc["std"].plot(kind='bar')
dsc.loc[pd.IndexSlice[:,'std'], :].describe().loc['std'].plot(secondary_y=True, c='red')

In [ ]:
grp = pd.concat([cl.pct_change(), cl_contract["remain_date"]], axis=1).dropna().groupby('remain_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'50%'], :].plot()
dsc.loc[pd.IndexSlice[:,'count'], :].plot()

In [ ]:
grp = pd.concat([ho.pct_change(), ho_contract["remain_date"]], axis=1).groupby('remain_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'std'], :].plot()

In [ ]:
grp = pd.concat([xb.pct_change(), xb_contract["remain_date"]], axis=1).groupby('remain_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'std'], :].plot()
dsc.loc[pd.IndexSlice[:,'count'], :].plot()

In [ ]:
grp = pd.concat([ng.pct_change(), ng_contract["remain_date"]], axis=1).groupby('remain_date')
dsc = grp.describe()
dsc.loc[pd.IndexSlice[:,'std'], :].plot()

In [ ]:
dsc.loc[pd.IndexSlice[:,'count'], :].plot()

In [ ]:
df = xb[['f2', 'f3','f4']]
df = df.pct_change()

df['remain_date'] = xb_contract['remain_date']
df['flag'] = (df['remain_date'] < 34) & (df['remain_date'] > 14) 


#((df['f2'] - df['f4'])*df['flag']).cumsum().plot()
((df['f3'] - df['f4'])*df['flag']).cumsum().plot()
((df['f3'] - df['f4'])).cumsum().plot()
#((df['f2'] - df['f3'])*df['flag']).cumsum().plot()


In [ ]:
df = xb.copy()
df = df.divide(df['f1'], axis=0)

df['remain_date'] = xb_contract['remain_date']
s = (df['remain_date'] < 34) & (df['remain_date'] > 14) 
s * 1
df = df.mul(s, axis=0)
df[['f2', 'f3', 'f4', 'f5',]].apply(max, axis=1)
df[['f2', 'f3', 'f4', 'f5',]].apply(min, axis=1)
df == df[['f2', 'f3', 'f4', 'f5',]].apply(min, axis=1)
# xb


In [ ]:
d = {"a":123, "b":456}
l = [{"a":123, "b":456}, {"a":123, "b":456}]
pd.DataFrame(l,)